In [1]:
import xarray as xr
import matplotlib.pylab as plt
import Rbeast as rb
import numpy as np
import cartopy.crs as ccrss
import cartopy.feature as cf
import matplotlib as mpl
import pandas as pd
import my_funs
from sklearn.experimental import enable_iterative_imputer
from sklearn.neighbors import BallTree, DistanceMetric
from causalimpact import CausalImpact
import pickle
from sklearn.impute import IterativeImputer
from dask.diagnostics import ProgressBar
from tqdm import tqdm
from scipy import stats
from joblib import Parallel, delayed

mpl.rcParams["mathtext.default"] = "regular"
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from my_funs import outliers_index
dir = "/data/home/hamiddashti/hamid/nasa_above/greeness/data/"

Initialization done!


/data/home/hamiddashti/hamid/nasa_above/greeness/codes/src/my_funs.py:9: UserWarning: Shapely 2.0 is installed, but because PyGEOS is also installed, GeoPandas will still use PyGEOS by default for now. To force to use and test Shapely 2.0, you have to set the environment variable USE_PYGEOS=0. You can do this before starting the Python process, or in your code before importing geopandas:

import os
os.environ['USE_PYGEOS'] = '0'
import geopandas

In a future release, GeoPandas will switch to using Shapely by default. If you are using PyGEOS directly (calling PyGEOS functions on geometries from GeoPandas), this will then stop working and you are encouraged to migrate from PyGEOS to Shapely 2.0 (https://shapely.readthedocs.io/en/latest/migration_pygeos.html).
  import geopandas as gpd
2023-08-12 06:32:35.745440: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the follo

In [2]:
n = 2000000
tmp = xr.open_dataarray(dir+"processed_data/landsat/mosaic/mosaic_NDVI_1984_TM5.tif",engine="rasterio").squeeze()
rng = np.random.default_rng(seed=2)
lats = []
lons=[]
for i in tqdm(range(n)):
    lat = rng.choice(tmp.shape[0], 1, replace=False)
    lon = rng.choice(tmp.shape[1], 1, replace=False)
    lats.append(lat)
    lons.append(lon)
lats = np.array(lats).squeeze()
lons = np.array(lons).squeeze()

100%|██████████| 2000000/2000000 [01:34<00:00, 21145.31it/s]


In [3]:
def var_extract(j,da):
    res = (da.isel(y=lats[j],x=lons[j]).values)
    return res

In [4]:
lc = []
da  = xr.open_dataarray(dir+"/raw_data/landcover/mosaic/mosaic_1984.tif")
years = np.arange(1984,2015)
for year in years:
    fname = dir+ "/raw_data/landcover/mosaic/mosaic_"+str(year)+".tif"
    da = xr.open_dataarray(fname,engine="rasterio").squeeze()
    print(fname)
    # for j in tqdm(range(len(lats))):
    #     ndvi.append(da.isel(y=lats[j],x=lons[j]).values)
    lc.append(Parallel(n_jobs=16)(delayed(var_extract)(j,da)
                            for j in range(len(lats))))
lc_array = np.array(lc).transpose()
with open('../data/lc_array.pickle', 'wb') as handle:
    pickle.dump(lc_array, handle, protocol=pickle.HIGHEST_PROTOCOL)

/data/home/hamiddashti/hamid/nasa_above/greeness/data//raw_data/landcover/mosaic/mosaic_1984.tif
/data/home/hamiddashti/hamid/nasa_above/greeness/data//raw_data/landcover/mosaic/mosaic_1985.tif
/data/home/hamiddashti/hamid/nasa_above/greeness/data//raw_data/landcover/mosaic/mosaic_1986.tif
/data/home/hamiddashti/hamid/nasa_above/greeness/data//raw_data/landcover/mosaic/mosaic_1987.tif
/data/home/hamiddashti/hamid/nasa_above/greeness/data//raw_data/landcover/mosaic/mosaic_1988.tif
/data/home/hamiddashti/hamid/nasa_above/greeness/data//raw_data/landcover/mosaic/mosaic_1989.tif
/data/home/hamiddashti/hamid/nasa_above/greeness/data//raw_data/landcover/mosaic/mosaic_1990.tif
/data/home/hamiddashti/hamid/nasa_above/greeness/data//raw_data/landcover/mosaic/mosaic_1991.tif
/data/home/hamiddashti/hamid/nasa_above/greeness/data//raw_data/landcover/mosaic/mosaic_1992.tif
/data/home/hamiddashti/hamid/nasa_above/greeness/data//raw_data/landcover/mosaic/mosaic_1993.tif
/data/home/hamiddashti/hamid/n

In [5]:
years = np.arange(1984,2015)
ndvi = []
for year in years:
    if year<=1999:
        fname = dir+"processed_data/landsat/mosaic/mosaic_NDVI_"+str(year)+"_TM5.tif"
        da = xr.open_dataarray(fname,engine="rasterio").squeeze()
        print(fname)
        # for j in tqdm(range(len(lats))):
        #     ndvi.append(da.isel(y=lats[j],x=lons[j]).values)
        ndvi.append(Parallel(n_jobs=16)(delayed(var_extract)(j,da)
                              for j in range(len(lats))))
        continue
    fname = dir+"processed_data/landsat/mosaic/mosaic_NDVI_"+str(year)+"_ETM.tif"
    da = xr.open_dataarray(fname,engine="rasterio").squeeze()
    print(fname)
    # for j in tqdm(range(len(lats))):
    #     ndvi.append(da.isel(y=lats[j],x=lons[j]).values)
    ndvi.append(Parallel(n_jobs=16)(delayed(var_extract)(j,da)
                              for j in range(len(lats))))
ndvi_array = np.array(ndvi).transpose()
with open('../data/ndvi_array.pickle', 'wb') as handle:
    pickle.dump(ndvi_array, handle, protocol=pickle.HIGHEST_PROTOCOL)

/data/home/hamiddashti/hamid/nasa_above/greeness/data/processed_data/landsat/mosaic/mosaic_NDVI_1984_TM5.tif
/data/home/hamiddashti/hamid/nasa_above/greeness/data/processed_data/landsat/mosaic/mosaic_NDVI_1985_TM5.tif
/data/home/hamiddashti/hamid/nasa_above/greeness/data/processed_data/landsat/mosaic/mosaic_NDVI_1986_TM5.tif
/data/home/hamiddashti/hamid/nasa_above/greeness/data/processed_data/landsat/mosaic/mosaic_NDVI_1987_TM5.tif
/data/home/hamiddashti/hamid/nasa_above/greeness/data/processed_data/landsat/mosaic/mosaic_NDVI_1988_TM5.tif
/data/home/hamiddashti/hamid/nasa_above/greeness/data/processed_data/landsat/mosaic/mosaic_NDVI_1989_TM5.tif
/data/home/hamiddashti/hamid/nasa_above/greeness/data/processed_data/landsat/mosaic/mosaic_NDVI_1990_TM5.tif
/data/home/hamiddashti/hamid/nasa_above/greeness/data/processed_data/landsat/mosaic/mosaic_NDVI_1991_TM5.tif
/data/home/hamiddashti/hamid/nasa_above/greeness/data/processed_data/landsat/mosaic/mosaic_NDVI_1992_TM5.tif
/data/home/hamiddas

In [6]:
years = np.arange(1984,2015)
nir = []
for year in years:
    if year<=1999:
        fname = dir+"processed_data/landsat/mosaic/mosaic_NIR_"+str(year)+"_TM5.tif"
        da = xr.open_dataarray(fname,engine="rasterio").squeeze()
        print(fname)
        # for j in tqdm(range(len(lats))):
        #     ndvi.append(da.isel(y=lats[j],x=lons[j]).values)
        nir.append(Parallel(n_jobs=16)(delayed(var_extract)(j,da)
                              for j in range(len(lats))))
        continue
    fname = dir+"processed_data/landsat/mosaic/mosaic_NIR_"+str(year)+"_ETM.tif"
    da = xr.open_dataarray(fname,engine="rasterio").squeeze()
    print(fname)
    # for j in tqdm(range(len(lats))):
    #     ndvi.append(da.isel(y=lats[j],x=lons[j]).values)
    nir.append(Parallel(n_jobs=16)(delayed(var_extract)(j,da)
                              for j in range(len(lats))))
nir_array = np.array(nir).transpose()
with open('../data/nir_array.pickle', 'wb') as handle:
    pickle.dump(nir_array, handle, protocol=pickle.HIGHEST_PROTOCOL)

/data/home/hamiddashti/hamid/nasa_above/greeness/data/processed_data/landsat/mosaic/mosaic_NIR_1984_TM5.tif
/data/home/hamiddashti/hamid/nasa_above/greeness/data/processed_data/landsat/mosaic/mosaic_NIR_1985_TM5.tif
/data/home/hamiddashti/hamid/nasa_above/greeness/data/processed_data/landsat/mosaic/mosaic_NIR_1986_TM5.tif
/data/home/hamiddashti/hamid/nasa_above/greeness/data/processed_data/landsat/mosaic/mosaic_NIR_1987_TM5.tif
/data/home/hamiddashti/hamid/nasa_above/greeness/data/processed_data/landsat/mosaic/mosaic_NIR_1988_TM5.tif
/data/home/hamiddashti/hamid/nasa_above/greeness/data/processed_data/landsat/mosaic/mosaic_NIR_1989_TM5.tif
/data/home/hamiddashti/hamid/nasa_above/greeness/data/processed_data/landsat/mosaic/mosaic_NIR_1990_TM5.tif
/data/home/hamiddashti/hamid/nasa_above/greeness/data/processed_data/landsat/mosaic/mosaic_NIR_1991_TM5.tif
/data/home/hamiddashti/hamid/nasa_above/greeness/data/processed_data/landsat/mosaic/mosaic_NIR_1992_TM5.tif
/data/home/hamiddashti/hamid

In [ ]:
#Sequential code
years = np.arange(1984,1985)
fname=[]
ndvi = []
for year in years:
    if year<=1999:
        fname = dir+"mosaic_NDVI_"+str(year)+"_TM5.tif"
        da = xr.open_dataarray(fname,engine="rasterio").squeeze()
        print(fname)
        for j in tqdm(range(len(lats))):
            ndvi.append(da.isel(y=lats[j],x=lons[j]).values)
        continue
    fname = dir+"mosaic_NDVI_"+str(year)+"_ETM.tif"
    da = xr.open_dataarray(fname,engine="rasterio").squeeze()
    print(fname)
    for j in tqdm(range(len(lats))):
        ndvi.append(da.isel(y=lats[j],x=lons[j]).values)
ndvi_array = np.array(ndvi).reshape(n,len(years),order="F")
with open('../../working/ndvi.pickle', 'wb') as handle:
    pickle.dump(ndvi_array, handle, protocol=pickle.HIGHEST_PROTOCOL)